In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
import random
import datetime

In [2]:
def splitSeconds(n, country, t):
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=True)
    samples = []
    for track in tracks:
        try:
            trackFeats = data[data.track_id == track]
            FeatsLen =  trackFeats.shape[0]
            #ind = np.random.choice(trackFeats.index[:-5], size=1, replace=True)
            ind = random.randrange(1, FeatsLen - 10)
            example = np.empty((0,24))
            while len(example) < w_length:
                feats = np.array(trackFeats.iloc[ind].loc["p1":"t12"])
                reps = int(trackFeats.iloc[ind].loc["duration"] * 100)
                if reps > 0:
                    hold = np.tile(feats, reps).reshape(reps, -1)
                    example = np.append(example, hold, axis = 0)
                ind = ind + 1
              #  example = np.array(example)
                example = example[-w_length:]
            samples = np.append(samples, example)
        except:
            continue
    samples = np.array(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [3]:
def getSamples(train_n, val_n):
    train = pd.DataFrame()
    train_labels = pd.DataFrame()
    val = pd.DataFrame()
    val_labels = pd.DataFrame()
    train_x = []
    train_labels = []
    val_x = []
    val_labels = []
    for country in countriesOfInterest:
        x1, y1 = splitSeconds(train_n, country, "train")
        x2, y2 = splitSeconds(val_n, country, "val")
        train_x = train_x + x1.tolist()
        train_labels = train_labels + y1.tolist()
        val_x = val_x + x2.tolist()
        val_labels = val_labels + y2.tolist()
    train_x = np.array(train_x)
    train_labels = np.array(train_labels)
    val_x = np.array(val_x)
    val_labels = np.array(val_labels)
    class_weights = class_weight.compute_class_weight('balanced',
                                                     np.unique(train_labels),
                                                     list(train_labels))
    train_labels = enc.transform(np.array(train_labels).reshape(-1,1)).toarray()
    val_labels = enc.transform(np.array(val_labels).reshape(-1,1)).toarray()
    return train_x, train_labels, val_x, val_labels, class_weights

In [ ]:
for i in range(500):
    countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
    countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BE", "CO"]
    enc = OneHotEncoder()
    enc.fit(np.array(countriesOfInterest).reshape(-1, 1))
    w_length = 200
    h = getSamples(500,100)
    pickle.dump( h, open( "Samples\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".p", "wb" ) )

In [ ]:
h = getSamples(500,100)
pickle.dump( h, open( "Samples\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".p", "wb" ) )